# Phrase Modelling Part III - Analyzing the Models

In [2]:
import os
from gensim import corpora, models, utils
from pprint import pprint
import pandas as pd
import time

## Load data

Load dictionary, corpus, and model

In [4]:
path = os.getcwd() + '/tmp_sample/'

allnews_dict = corpora.Dictionary.load(path + 'allnews_UPOSfilter_dict.dict')
allnews_corpus = corpora.MmCorpus(path + 'allnews_UPOSfilter_corpus.mm')
allnews_model = models.ldamodel.LdaModel.load(path + 'allnews_UPOSfilter_lda_model_10')

path = os.getcwd() + '/csv/'

trigram_cols = [str(i) for i in list(range(134))]
allnews_trigrams = pd.read_csv(path + 'allnews_UPOSfilter_addtrigrams.csv', usecols=trigram_cols).values.tolist()

/usr/local/miniconda3/envs/nlp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
allnews_trigrams[:3]

[['says',
  'misdiagnosed',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 ['plunges',
  nan,
  nan,
  nan,
  nan,
  nan,

In [6]:
pprint(allnews_model.print_topics())

[(0,
  '0.115*"new" + 0.018*"watch" + 0.015*"old" + 0.014*"found" + 0.014*"urges" + '
  '0.013*"wants" + 0.013*"live" + 0.012*"announces" + 0.011*"remove" + '
  '0.010*"launches"'),
 (1,
  '0.159*"says" + 0.029*"warns" + 0.018*"national" + 0.013*"biggest" + '
  '0.012*"low" + 0.012*"wear" + 0.011*"seeks" + 0.011*"makes" + 0.010*"small" '
  '+ 0.010*"faces"'),
 (2,
  '0.098*"best" + 0.085*"russian" + 0.034*"latest" + 0.031*"passes" + '
  '0.027*"build" + 0.025*"based" + 0.023*"start" + 0.023*"begins" + '
  '0.022*"leave" + 0.019*"kills"'),
 (3,
  '0.048*"global" + 0.031*"hit" + 0.029*"economic" + 0.028*"sees" + '
  '0.027*"dead" + 0.026*"agricultural" + 0.023*"cut" + 0.019*"online" + '
  '0.019*"german" + 0.017*"open"'),
 (4,
  '0.066*"pandemic" + 0.063*"chinese" + 0.057*"european" + 0.027*"led" + '
  '0.027*"nearly" + 0.026*"going" + 0.025*"leads" + 0.024*"worst" + '
  '0.023*"italian" + 0.022*"needs"'),
 (5,
  '0.051*"set" + 0.041*"american" + 0.038*"calls" + 0.032*"big" + 0.031*"gets

In [7]:
start = time.time()

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(allnews_model, allnews_corpus, allnews_trigrams)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

end = time.time()
print("Elapsed:", end - start, 'seconds')

Elapsed: 9612.574942350388 seconds


In [8]:
len(df_dominant_topic)

699176

In [9]:
df_dominant_topic.head(100)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.1888,"says, warns, national, biggest, low, wear, see...","[says, misdiagnosed, nan, nan, nan, nan, nan, ..."
1,1,1.0,0.1668,"says, warns, national, biggest, low, wear, see...","[plunges, nan, nan, nan, nan, nan, nan, nan, n..."
2,2,1.0,0.2018,"says, warns, national, biggest, low, wear, see...","[mean, american, bad, nan, nan, nan, nan, nan,..."
3,3,1.0,0.1632,"says, warns, national, biggest, low, wear, see...","[plunges, disappoints, nan, nan, nan, nan, nan..."
4,4,1.0,0.1668,"says, warns, national, biggest, low, wear, see...","[suspended, nan, nan, nan, nan, nan, nan, nan,..."
...,...,...,...,...,...
95,95,1.0,0.1770,"says, warns, national, biggest, low, wear, see...","[federal, blocks, near, total, nan, nan, nan, ..."
96,96,1.0,0.1632,"says, warns, national, biggest, low, wear, see...","[rate, fast, produced, nan, nan, nan, nan, nan..."
97,97,1.0,0.1668,"says, warns, national, biggest, low, wear, see...","[increase, nan, nan, nan, nan, nan, nan, nan, ..."
98,98,1.0,0.1632,"says, warns, national, biggest, low, wear, see...","[driving, die, nan, nan, nan, nan, nan, nan, n..."


In [10]:
df_dominant_topic.Dominant_Topic.value_counts()

1.0    547326
0.0    136033
8.0     14565
3.0      1237
9.0        12
7.0         1
4.0         1
5.0         1
Name: Dominant_Topic, dtype: int64

In [11]:
start = time.time()

# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

end = time.time()
print("Elapsed:", end - start, 'seconds')

Elapsed: 0.2416832447052002 seconds


In [12]:
sent_topics_sorteddf_mallet


,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0.0,0.2832,"new, watch, old, found, urges, wants, live, announces, remove, launches","[old, accused, killing, found, beaten, slashed, partially, bound, nan, nan, nan, nan, nan, nan, ..."
1,1.0,0.2540,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[scared, catch, serving, prison, says, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na..."
2,3.0,0.1975,"global, hit, economic, sees, dead, agricultural, cut, online, german, open","[searching, dead, extra, largest, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na..."
3,4.0,0.1639,"pandemic, chinese, european, led, nearly, going, leads, worst, italian, needs","[democratic, presidential, democratic, presidential, demand, toss, current, nan, nan, nan, nan, ..."
4,5.0,0.1719,"set, american, calls, big, gets, public, face, real, likely, nuclear","[getting, desperately, needed, knocked, fires, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
5,7.0,0.1627,"close, federal, major, dies, financial, local, long, better, use, corporate","[ok, feel, ok, right, feel, better, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
6,8.0,0.2248,"second, social, killed, help, want, bans, shows, stop, record, primary","[killed, shooting, outside, appeared, targeted, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan..."
7,9.0,0.1755,"black, need, right, rise, buy, takes, rising, white, vote, far","[claims, override, different, allow, reopen, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n..."


In [13]:
path = os.getcwd() + '/csv/'
allnews_csv = pd.read_csv(path + 'allnews_UPOSfilter_addtrigrams.csv')

/usr/local/miniconda3/envs/nlp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
consolidated_csv = pd.concat([allnews_csv, df_dominant_topic], axis=1)

In [15]:
path = os.getcwd() + '/csv/'

consolidated_csv.to_csv(path + 'allnews_UPOSfilter_consolidated_10.csv')

In [16]:
consolidated_csv.columns.tolist()

['Unnamed: 0',
 'datetime2',
 'url',
 'source_x',
 'title',
 'care_p',
 'fairness_p',
 'loyalty_p',
 'authority_p',
 'sanctity_p',
 'care_sent',
 'fairness_sent',
 'loyalty_sent',
 'authority_sent',
 'sanctity_sent',
 'total_words',
 'datetime2.1',
 'care_prod',
 'fairness_prod',
 'loyalty_prod',
 'authority_prod',
 'sanctity_prod',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '9

In [17]:
cols = ['datetime2','title',
        'care_prod','fairness_prod','loyalty_prod','authority_prod','sanctity_prod',
       'Dominant_Topic','Topic_Perc_Contrib','Keywords','Text']

path = os.getcwd() + '/csv/'
consolidated_csv[cols].to_csv(path + 'allnews_UPOSfilter_consolidated_for_anova_10.csv')

In [18]:
consolidated_csv[cols][:50]

,datetime2,title,care_prod,fairness_prod,loyalty_prod,authority_prod,sanctity_prod,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,2019-10-31 19:00:00-04:00,"Trent Williams, Redskins OT, says he had cancer, team misdiagnosed it",-0.466280,-0.091422,-0.135952,-0.311245,-0.079818,1.0,0.1888,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[says, misdiagnosed, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,2019-10-31 19:00:00-04:00,Pinterest Stock Plunges on Sales Miss,0.232462,-0.278476,-2.041658,-2.859952,-0.035497,1.0,0.1668,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[plunges, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n..."
2,2019-10-31 19:00:00-04:00,Baghdadi's death could mean more American withdrawal in the Middle East. That's bad news.,-0.069877,0.108114,-0.178720,-0.028491,0.020125,1.0,0.2018,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[mean, american, bad, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
3,2019-10-31 19:00:00-04:00,Arista Networks Stock Plunges as Earnings Outlook Disappoints,-0.755164,-0.212345,-0.535497,-0.139791,-0.584116,1.0,0.1632,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[plunges, disappoints, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan..."
4,2019-10-31 19:00:00-04:00,"Joel Embiid, Karl-Anthony Towns suspended two games by NBA after fight",-0.032744,0.550585,1.926355,0.756060,0.537998,1.0,0.1668,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[suspended, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
5,2019-10-31 19:00:00-04:00,Heidi Klum's 2019 Halloween costume transformation is scary good,-1.798600,-1.151093,-0.674112,0.087814,-0.087629,0.0,0.1704,"new, watch, old, found, urges, wants, live, announces, remove, launches","[scary, good, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na..."
6,2019-10-31 19:00:00-04:00,The Epic Macklowe Divorce: How A Famous $1 Billion Art Collection Got Snagged In Court,-0.948759,-0.542771,-0.754931,0.204665,0.011230,1.0,0.1706,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[snagged, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n..."
7,2019-10-31 19:00:00-04:00,D.C. passes the nation's report card. So why would the city roll back education reform?,-0.045958,0.962118,0.952784,0.545746,-0.071214,1.0,0.1632,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[passes, roll, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n..."
8,2019-10-31 19:00:00-04:00,Texas man convicted of fatally stabbing stepmother who danced after Cowboys victory,-0.099868,-1.196121,0.224584,0.062751,-0.212859,1.0,0.1632,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[convicted, fatally, danced, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na..."
9,2019-10-31 19:00:00-04:00,Was David Montgomery over Jordan Howard a win for Chicago Bears?,-0.170139,-0.707600,0.007451,0.182855,-0.442433,1.0,0.1706,"says, warns, national, biggest, low, wear, seeks, makes, small, faces","[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."


In [19]:
consolidated_csv[cols].shape

(699176, 11)